<a href="https://colab.research.google.com/github/ratm123/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/AG3/Jose_Luis_Llinares_Anton_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


AG3 - Actividad guiada 3

Jose Luis Llinares Anton

https://github.com/ratm123/03MAIR---Algoritmos-de-Optimizacion---2019


https://colab.research.google.com/drive/14JX5VTDhl295aYHsaifsCkqgMW6VfSLc

In [0]:
!pip install requests


In [0]:
import urllib.request


file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f69ab7680f0>)

In [0]:
!pip install tsplib95

In [0]:
import tsplib95
import random
import math


In [0]:
problem = tsplib95.load_problem(file)

nodos = list(problem.get_nodes())
aristas = list(problem.get_edges())

## Funciones generales

In [0]:
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.wfunc(a,b)

#Devuelve la distancia total de una trayectoria
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)


solucion = crear_solucion(nodos)
distancia_total(solucion, problem)

5093

## Busqueda aleatoria

In [0]:
def busqueda_aleatoria(problem, N):
  mejor_solucion = crear_solucion(nodos)
  mejor_valor = distancia_total(mejor_solucion, problem)
  for i in range(N):
    solucion = crear_solucion(nodos)
    valor = distancia_total(solucion, problem)
    if valor < mejor_valor:
      mejor_solucion = solucion
      mejor_valor = valor

  print("La mejor solucion es:",mejor_solucion, " y  el mejor valor:", mejor_valor)

busqueda_aleatoria(problem, 1000)

La mejor solucion es: [0, 39, 14, 3, 6, 9, 12, 25, 11, 2, 21, 24, 40, 22, 27, 4, 7, 26, 10, 23, 5, 36, 20, 28, 29, 13, 18, 1, 33, 31, 35, 17, 16, 41, 19, 15, 37, 38, 30, 32, 8, 34]  y  el mejor valor: 3669


## Busqueda local

In [0]:
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] 
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

#####################################################################
def busqueda_local(problem, N):
  #problem = datos del problema
  #N = numero de iteraciones
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  solucion_referencia = crear_solucion(nodos)
  
  for i in range(N):
    vecina = genera_vecina(solucion_referencia)
    distancia_vecina = distancia_total(vecina, problem)
    if distancia_vecina <= mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
        
    solucion_referencia = vecina
  print("mejor solucion:", mejor_distancia,  mejor_solucion)

busqueda_local(problem, 100)

mejor solucion: 1668 [0, 1, 7, 37, 17, 31, 3, 27, 30, 38, 22, 39, 24, 40, 21, 29, 28, 2, 4, 15, 16, 14, 19, 13, 18, 10, 8, 9, 23, 41, 25, 11, 12, 26, 5, 6, 32, 34, 33, 20, 35, 36]


In [0]:
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  if j<i:
    i,j=j,i
  vecina = list(solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:])
  return vecina

solucion = crear_solucion(nodos)
print(genera_vecina_aleatorio(genera_vecina_aleatorio(solucion)))

[0, 2, 13, 15, 23, 16, 35, 33, 17, 14, 12, 18, 30, 41, 19, 11, 32, 21, 1, 4, 25, 39, 10, 37, 36, 24, 27, 31, 29, 5, 8, 7, 38, 28, 9, 26, 3, 6, 34, 22, 20, 40]


## Recocido simulado - SA

In [0]:
#Funcion de probabilidad para determinar si se cambia 
#    a una solución peor respecto a la de referencia(exponencial)
def probabilidad(T,d):
  if random.random() <= math.exp(-1*d / T)   :
    return True
  else:
    return False


def bajar_temperatura(T):
  return T*.9999

In [0]:
#####################################################################
def recocido_simulado(problem, TEMPERATURA=1000):
  #problem = datos del problema

  solucion_referencia = crear_solucion(nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
  while TEMPERATURA > 1:
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina_aleatorio(solucion_referencia) 
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      print('\t****Encontrada mejor solucion:', mejor_distancia , mejor_solucion, '\n')
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según 
    # una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    p = probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina))
    if distancia_vecina < distancia_referencia or p :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
    
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es :" , mejor_distancia, mejor_solucion)
  return mejor_solucion

sol = recocido_simulado(problem, TEMPERATURA=1000)


	****Encontrada mejor solucion: 4948 [0, 17, 3, 31, 5, 25, 8, 38, 6, 26, 41, 20, 2, 10, 19, 21, 22, 23, 1, 4, 13, 9, 28, 12, 35, 34, 7, 16, 39, 11, 30, 36, 27, 15, 18, 14, 40, 37, 29, 24, 32, 33] 

	****Encontrada mejor solucion: 4866 [0, 17, 3, 31, 5, 21, 8, 38, 19, 26, 23, 20, 2, 10, 1, 25, 24, 27, 6, 4, 13, 9, 40, 7, 35, 34, 22, 16, 12, 11, 30, 36, 18, 15, 41, 14, 28, 37, 29, 39, 32, 33] 

	****Encontrada mejor solucion: 4699 [0, 17, 16, 31, 5, 38, 8, 21, 19, 26, 23, 20, 2, 10, 1, 25, 24, 27, 6, 4, 13, 9, 40, 7, 35, 34, 22, 3, 12, 11, 30, 36, 18, 15, 41, 14, 28, 37, 29, 39, 32, 33] 

	****Encontrada mejor solucion: 4697 [0, 17, 16, 31, 5, 38, 8, 21, 19, 30, 23, 20, 2, 10, 1, 25, 24, 27, 6, 4, 13, 9, 40, 7, 35, 34, 22, 3, 12, 11, 26, 36, 18, 15, 41, 14, 28, 37, 29, 39, 32, 33] 

	****Encontrada mejor solucion: 4445 [0, 17, 16, 31, 5, 38, 8, 21, 40, 30, 23, 20, 2, 10, 1, 25, 24, 27, 6, 4, 13, 9, 19, 7, 35, 34, 22, 3, 12, 11, 26, 36, 18, 15, 41, 14, 28, 37, 29, 39, 32, 33] 

	****Encon

## Colonia de hormigas

In [0]:
def Add_Nodo(problem, H ,T ) :
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )


def Incrementa_Feromona(problem, T, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(nodos)) ] for j in range(len(nodos))]
  return T

def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
    #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]


  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #print("\nAgente:", h)
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
  
  print(mejor_solucion)
  print(mejor_distancia)

hormigas(problem, 100)

[0, 7, 3, 14, 34, 38, 33, 29, 1, 36, 16, 35, 5, 41, 6, 15, 19, 8, 9, 24, 12, 13, 31, 20, 2, 17, 32, 28, 40, 26, 25, 23, 10, 37, 18, 11, 30, 27, 4, 22, 21, 39]
3943
